# Using two cities code

In [6]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
import json
from tqdm import tqdm_notebook as tqdm
import gmplot
from pprint import pprint
import gmaps
# Google developer API key
from config import gkey
from config2 import geokey
# Configure gmaps
gmaps.configure(api_key=gkey)
from opencage.geocoder import OpenCageGeocode

In [8]:
#Ask the user where he/she would like to travel
question0 = ("What is your departure city (Houston)? ")
departure_city = input(question0)
print("")
question1 =("Name one city that you would like to go on vacation (Paris)? ")
vacation_city = input(question1)
print("")
question1a =("Now name your second option of city that you would like to go on vacation (Newcastle)? ")
vacation_citya = input(question1a)
print("")
question2 =("What is your desire departure date (yyyy-mm-dd)? ")
departure_date = input(question2)
print("")
question3 =("What is your desire return date (yyyy-mm-dd)? ")
return_date = input(question3)

#print(f"-----------------&&&--------------------------")
print("")
#print(f"Thank you for selecting QUEST FLIGHTS. We are starting the search of best quotes: Flying from {departure_city} to {vacation_city}")
print("")
#print(f"and {vacation_citya} on {departure_date} and returning on {return_date}..")


############################### Getting first travel information #################################

combined_cities = [departure_city,vacation_city, vacation_citya]

cities_name = []
country_name = []
airportID =[]
airportName = []

#City counter kick off. 
city_counter = 1

#Base url
url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/USA/USD/en-US/?query="

#Print statement to inform when data acquisition start
print(f"Beginning Data Acquisition")
print(f"------------&&&&-------------------")

for i in tqdm(combined_cities):
    try:
        #Getting the API with the first outbound city in the list (Houston) and loop thru the list
        response = requests.get(url+i,
          headers={
            "X-RapidAPI-Host": "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
            "X-RapidAPI-Key": "09ec0160admsh2d5563a7feaa3efp1e2db5jsn5787845ed7fc"
          }
    ).json()
        #Export the name and the ID
        cities_name.append(response["Places"][0]["PlaceName"])
        country_name.append(response["Places"][0]["CountryName"])
        airportID.append(response["Places"][1]["PlaceId"])
        airportName.append(response["Places"][1]["PlaceName"])
        current_city = response["Places"][0]["PlaceName"]
        
        print(f"Data Acquisition {city_counter} | City: {current_city}")  
        city_counter = city_counter + 1
    except:
        print("Route not found...Try another vacation city")
    continue
    
# Print statement to inform when all records have been acquired.
print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")

#Creating data frame of first output
vacation = {'City Name': cities_name, 'Country Name': country_name, 'Airport Name':airportName, 'Airport Code': airportID}
vacation_df = pd.DataFrame(vacation)
vacation_df.rename(index={0:'From'}, inplace=True)
vacation_df.rename(index={1:'To'}, inplace=True)
vacation_df
vacation_airport_codestolist = vacation_df['Airport Code'].tolist()
vacation_airport_codestolist[0]
########################################## World Map #######################################

#https://amaral.northwestern.edu/blog/getting-long-lat-list-cities
#https://opencagedata.com/tutorials/geocode-in-python
key = geokey

geocoder = OpenCageGeocode(key)

list_lat = []
list_long = []

print("")
print(f"Beginning Data Acquisition of Destinations Map")
print(f"------------&&&&-------------------")

for index, row in tqdm(vacation_df.iterrows()): #iterate over rows in data frame
    City = row['City Name']
    Country = row['Country Name']
    query = str(City)+','+str(Country)
    
    results = geocoder.geocode(query)
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']
    
    list_lat.append(lat)
    list_long.append(long)

print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")
print("")
# create a new columns from lists

vacation_df['lat'] = list_lat
vacation_df['long'] = list_long

# Store latitude and longitude in locations
locations = vacation_df[["lat", "long"]]
locations  
# Plot Markers MAp
fig = gmaps.figure()

# Create heat layer
markers = gmaps.marker_layer(locations)


# Add layer
fig.add_layer(markers)

# Display figure
fig



What is your departure city (Houston)? Houston

Name one city that you would like to go on vacation (Paris)? Paris

Now name your second option of city that you would like to go on vacation (Newcastle)? Newcastle

What is your desire departure date (yyyy-mm-dd)? 2019-12-10

What is your desire return date (yyyy-mm-dd)? 2020-01-10


Beginning Data Acquisition
------------&&&&-------------------


Data Acquisition 1 | City: Houston
Data Acquisition 2 | City: Paris
Data Acquisition 3 | City: Newcastle

------------&&&&---------------
Data Acquisition Complete
------------&&&&---------------

Beginning Data Acquisition of Destinations Map
------------&&&&-------------------



------------&&&&---------------
Data Acquisition Complete
------------&&&&---------------



Figure(layout=FigureLayout(height='420px'))

In [9]:

################################## Acquiring Data to your Top 2 cities ################################3

# Creating list to hold values of the API results

carriers_info = []
places_info = []
quotes_info = []

base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browseroutes/v1.0/US/USD/en-US/"

headers={"X-RapidAPI-Host": "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    "X-RapidAPI-Key": "077cfd7813mshfe97bf766bd036dp106fd9jsneb6caf492da9"}

originplace = vacation_airport_codestolist
destinationplace = vacation_airport_codestolist
outboundpartialdate = departure_date
inboundpartialdate = return_date

destination_counter = 1
#Print statement to inform when data acquisition start
print(f"Beginning Data Acquisition for you trip from {vacation_airport_codestolist[0]} to your top two options !!!")
print(f"------------&&&&-------------------")

for x in tqdm(destinationplace):
    try:
        response = requests.get(base_url+vacation_airport_codestolist[0]+"/"+x+ "/"+departure_date+"?inboundpartialdate="+return_date, headers=headers).json()
        carriers_info.append(response['Carriers'])
        places_info.append(response['Places'])
        quotes_info.append(response['Quotes'])
        
        #print(f"Data Acquisition {destination_counter} | Cities: {vacation_airport_codestolist[0]}")  
        
        destination_counter = destination_counter + 1
    except:
         print("Route not found...!!")
    continue
    
# Print statement to inform when all records have been acquired.
print(f"------------&&&&---------------")
print(f"Data Acquisition Complete")
print(f"------------&&&&---------------")
#print(json.dumps(carriers_info,indent=4, sort_keys=True))
#print(json.dumps(places_info,indent=4, sort_keys=True))
#print(json.dumps(quotes_info,indent=4, sort_keys=True))   

####################### Looping ###################################33

#Loop thru the carriers information data that contains independent dictionaries and append results to a empty list
carriers_list = []
for x in range(len(carriers_info)):
    for y in range(len(carriers_info[x])):
        #print(carriers_info[x][y])
        carriers_list.append(carriers_info[x][y])
        

#Loop thru the places information data that contains dictionaries and append results to a empty list
places_list = []
for i in range(len(places_info)):
    for j in range(len(places_info[i])):
        #print(places_info[i][j])
        places_list.append(places_info[i][j])

#Loop thru the quoute information data that contains nested dictionaries and append results to a empty list
#Noted the nested dictionary of the outboundleg which is append to the outbound list
quotes_list = []
outbound_list = []
for z in range(len(quotes_info)):
    for w in range(len(quotes_info[z])):
        quotes_list.append(quotes_info[z][w])
        outbound_list.append(quotes_info[z][w]['OutboundLeg'])   

#Create the data frame of the quotes list
quotes_df = pd.DataFrame(quotes_list)
quotes_df.head()

#Create the data frame of the outbound list
outbound_df = pd.DataFrame(outbound_list)
outbound_df

#Merged both data frames together on the index since data is symetric
merged_quotes_outbound = pd.merge(quotes_df,outbound_df, left_index = True, right_index = True)
merged_quotes_outbound

################################## Export to CSV ######################################################
#Export to CSV 
export_csv = merged_quotes_outbound.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\Test_merged_quotes_outbound.csv',index = False)
merged_quotes_outbound        
        
#Creating the data frame for places
places_df = pd.DataFrame(places_list)
places_df
#Exporting dataframe into csv format
export_csv = places_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\Testplacescsv.csv',index = False)

#Creating the data frame of carrriers
carriers_df = pd.DataFrame(carriers_list)
carriers_df
#Exporting dataframe into csv format
export_csv = carriers_df.to_csv(r'C:\Users\pablo\Google Drive\Certifications\Rice Data Analytics\HomeWorks_Projects\COPY_Project_1\Resources\Test_carriers_df.csv',index = False)







Beginning Data Acquisition for you trip from IAH-sky to your top two options !!!
------------&&&&-------------------



------------&&&&---------------
Data Acquisition Complete
------------&&&&---------------


# Using one city Code 

In [ ]:
# #Ask the user where he/she would like to travel
# question0 = ("What is your departure city? ")
# departure_city = input(question0)

# question1 =("Where would you like to go on vacation? ")
# vacation_city = input(question1)

# question2 =("What is your desire departure date (yyyy-mm-dd)? ")
# departure_date = input(question2)

# question3 =("What is your desire return date (yyyy-mm-dd)? ")
# return_date = input(question3)

# print(f"-----------------&&&--------------------------")
# print(f"Thank you for selecting QUEST FLIGHTS. We are starting the search of best quotes: Flying from {departure_city} to {vacation_city} on {departure_date} and returning on {return_date}..")

# #URL set to the get outbound and destination airport key
# combined_cities = [departure_city,vacation_city]

# cities_name = []
# airportID =[]
# airportName = []

# #City counter kick off. 
# city_counter = 1

# #Base url
# url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/USA/USD/en-US/?query="

# #Print statement to inform when data acquisition start
# print(f"Beginning Data Acquisition")
# print(f"------------&&&&-------------------")

# for i in tqdm(combined_cities):
#     try:
#         #Getting the API with the first outbound city in the list (Houston) and loop thru the list
#         response = requests.get(url+i,
#           headers={
#             "X-RapidAPI-Host": "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
#             "X-RapidAPI-Key": "09ec0160admsh2d5563a7feaa3efp1e2db5jsn5787845ed7fc"
#           }
#     ).json()
#         #Export the name and the ID
#         cities_name.append(response["Places"][0]["PlaceName"])
#         airportID.append(response["Places"][1]["PlaceId"])
#         airportName.append(response["Places"][1]["PlaceName"])
#         current_city = response["Places"][0]["PlaceName"]
        
#         print(f"Data Acquisition {city_counter} | City: {current_city}")  
#         city_counter = city_counter + 1
#     except:
#         print("Route not found...Try another vacation city")
#     continue
    
# # Print statement to inform when all records have been acquired.
# print(f"------------&&&&---------------")
# print(f"Data Acquisition Complete")
# print(f"------------&&&&---------------")

# #Creating data frame of first output
# vacation = {'City Name': cities_name, 'Airport Name':airportName, 'Airport Code': airportID}
# vacation_df = pd.DataFrame(vacation)
# vacation_df.rename(index={0:'From'}, inplace=True)
# vacation_df.rename(index={1:'To'}, inplace=True)
# vacation_df
# vacation_airport_codestolist = vacation_df['Airport Code'].tolist()
# vacation_airport_codestolist[0]
# return_date

# # Creating list to hold values of the API results

# carriers_info = []
# places_info = []
# quotes_info = []

# base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browseroutes/v1.0/US/USD/en-US/"

# headers={"X-RapidAPI-Host": "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
#     "X-RapidAPI-Key": "077cfd7813mshfe97bf766bd036dp106fd9jsneb6caf492da9"}

# originplace = vacation_airport_codestolist[0]
# destinationplace = vacation_airport_codestolist[1]
# outboundpartialdate = departure_date
# inboundpartialdate = return_date

# destination_counter = 1
# #Print statement to inform when data acquisition start
# #print(f"Beginning Data Acquisition")
# #print(f"------------&&&&-------------------")

# #for x in tqdm(destinationplace):
# try:
#     response = requests.get(base_url+originplace+"/"+destinationplace+ "/"+departure_date+"?inboundpartialdate="+return_date, headers=headers).json()
#     carriers_info.append(response['Carriers'])
#     places_info.append(response['Places'])
#     quotes_info.append(response['Quotes'])
        
#   #      print(f"Data Acquisition {destination_counter} | Outbound City: {vacation_airport_codestolist[0]}")  
        
#    #     destination_counter = destination_counter + 1
# except:
#          print("Route not found...!!")
#     #continue
    
# # Print statement to inform when all records have been acquired.
# #print(f"------------&&&&---------------")
# #print(f"Data Acquisition Complete")
# #print(f"------------&&&&---------------")
# print(json.dumps(carriers_info,indent=4, sort_keys=True))
# print(json.dumps(places_info,indent=4, sort_keys=True))
# print(json.dumps(quotes_info,indent=4, sort_keys=True))       